In [1]:
import sys
import os

# 프로젝트 루트 추가
project_root = os.path.dirname(os.path.abspath("__file__")) if "__file__" in globals() else os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import time
import functools
def benchmark_runtime_decorator(func):
    @functools.wraps(func)
    def wrapper_benchmark_runtime(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        elapsed_minute = int(elapsed_time / 60) if elapsed_time >= 60 else None
        print(f"Function '{func.__name__}' executed in {f"{elapsed_minute} minutes " if elapsed_minute else ""}{elapsed_time:.4f} seconds")
        return result

    return wrapper_benchmark_runtime

In [2]:
from common.Generator import Generator
from common.test_backendclient import TestBackendClient as BackendClient
import logging

# 기존 핸들러 제거
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# 기본 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
    ]
)

# 루트 로거 레벨
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

backend_client = BackendClient()
generator = Generator()

generator._test_pdf_create(backend_client)

c:\Users\KISIA\AppData\Local\Programs\Python\Python313\Lib\site-packages\pydantic\_internal\_generate_schema.py:918: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `Settings` to V2.
  warn(
c:\Users\KISIA\AppData\Local\Programs\Python\Python313\Lib\site-packages\pydantic\_internal\_generate_schema.py:918: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `Settings` to V2.
  warn(


Graph compiled successfully!
2025-10-24 12:09:29,082 - common.test_backendclient - INFO - Initialized TestBackendClient
2025-10-24 12:09:29,084 - common.Generator - INFO - PDF 생성 테스트 시작
2025-10-24 12:09:29,085 - common.Generator - DEBUG - Loading job info for task_id: test_task_id, job_id: test_job_id
2025-10-24 12:09:29,086 - common.test_backendclient - INFO - [Test] load_job_info called for task test_task_id, job test_job_id
2025-10-24 12:09:29,087 - common.Generator - DEBUG - Creating test report for user_id: e133aded-a617-4fc8-83f0-6cd05cd4464b
2025-10-24 12:09:29,088 - common.test_backendclient - INFO - [Test] save_report called for task test_id
2025-10-24 12:09:29,089 - common.test_backendclient - DEBUG - {'title': '정보 유출 진단 보고서', 'summary': '[의뢰 회사 이름]', 'pc_id': 'WORK-PC-001', 'task_id': 'test_id', 'details': [{'section_type': <SectionTypeEnum.analysis_objective: 0>, 'content': '본 분석의 목적은 대상 PC 로부터 사내 주요 기밀정보(고객정보, 사내 데이터 등)의 유출 정황을 파악하는 것입니다.', 'order_no': None}, {'section_typ

In [3]:
from common.Generator import Generator
from common.test_backendclient import TestBackendClient as BackendClient
from test.test_data_loader import test_data_loader_v1


backend_client = BackendClient()
generator = Generator()

task_id = "session-20251002-052932-151e52e9" # 10만개 데이터
# task_id = "session-20251002-040520-28a9dc6c" # 정호주임 pc

job_id = "73c94dcd-d1a1-41c4-9636-dd24816dfd9a"

@benchmark_runtime_decorator
def load_artifacts():
    artifacts = test_data_loader_v1(task_id)
    print(f"\n✅ 총 {len(artifacts):,}개 아티팩트 로드 완료")
    return artifacts

artifacts = load_artifacts()

job_info = backend_client.load_job_info(task_id, job_id)
print(job_info)

2025-10-24 12:09:31,869 - common.test_backendclient - INFO - Initialized TestBackendClient
2025-10-24 12:09:31,872 - test.testAnalyzer - INFO - 🚀 [TEST] Starting filtering test for task: session-20251002-052932-151e52e9
[2025-10-24 12:09:39] DataEncoder - Converted WebKit urls.last_visit_time to datetime
[2025-10-24 12:09:39] DataEncoder - Converted WebKit visits.visit_time to datetime
[2025-10-24 12:09:39] DataEncoder - Converted WebKit downloads.start_time to datetime
[2025-10-24 12:09:39] DataEncoder - Converted WebKit downloads.end_time to datetime
[2025-10-24 12:09:39] DataEncoder - Converted WebKit downloads.last_access_time to datetime
[2025-10-24 12:09:39] DataEncoder - Converted WebKit logins.date_created to datetime
[2025-10-24 12:09:39] DataEncoder - Converted WebKit logins.date_last_used to datetime
[2025-10-24 12:09:39] DataEncoder - Converted WebKit logins.date_received to datetime
[2025-10-24 12:09:39] DataEncoder - Converted WebKit logins.date_password_modified to datet

In [4]:
@benchmark_runtime_decorator
def generate_report(generator):
    generator.generate_report(task_id, job_id, artifacts, job_info, backend_client)

generate_report(generator)

2025-10-24 12:09:57,515 - common.Generator - INFO - Starting report generation for task_id: session-20251002-052932-151e52e9, job_id: 73c94dcd-d1a1-41c4-9636-dd24816dfd9a
2025-10-24 12:09:57,516 - common.Generator - DEBUG - Invoking LLM service to generate scenarios for task_id: session-20251002-052932-151e52e9

🔄 필터링 반복 1/3: VERY_STRICT
  - 입력: 원본 아티팩트
  - 현재 아티팩트: 52,991개
  - 필터링 강도: very_strict
  - 목표 비율: 1.5%
  - 목표 개수: 100,000개
  - 총 청크 수: 177개

📦 총 9개 배치로 처리

📦 배치 1/9 (청크 1-20)...
✅ 청크 2: 2개 발견
✅ 청크 4: 29개 발견
✅ 청크 1: 117개 발견
✅ 청크 3: 87개 발견
✅ 청크 5: 157개 발견
✅ 청크 8: 3개 발견
✅ 청크 9: 17개 발견
✅ 청크 6: 231개 발견
✅ 청크 12: 5개 발견
✅ 청크 7: 292개 발견
✅ 청크 10: 245개 발견
✅ 청크 11: 123개 발견
✅ 청크 13: 89개 발견
✅ 청크 14: 57개 발견
✅ 청크 17: 19개 발견
✅ 청크 18: 58개 발견
✅ 청크 15: 300개 발견
✅ 청크 20: 20개 발견
✅ 청크 16: 279개 발견
✅ 청크 19: 300개 발견
  ✅ 배치 1 완료: 2430개 발견
📦 배치 2/9 (청크 21-40)...
✅ 청크 21: 28개 발견
✅ 청크 23: 16개 발견
✅ 청크 22: 77개 발견
✅ 청크 26: 6개 발견✅ 청크 24: 168개 발견

✅ 청크 27: 17개 발견
✅ 청크 25: 300개 발견
✅ 청크 28: 14개 발견
✅ 청크 30: 4개 발견
✅ 

In [5]:
for item in generator.report:
    if item[0] == 'details':
        title, data = item
        for i in data:
            print(i.section_type.value, i.content)
        continue
    print(item)

('title', '윈도우 아티팩트 기반 정보 유출 진단 보고서')
('summary', '내부 기밀 문서 유출 가능성 존재함')
('pc_id', 'MAC_00-0C-29-EB-B3-7B')
('task_id', 'session-20251002-052932-151e52e9')
0 본 분석의 목적은 대상 PC 로부터 사내 주요 기밀정보(고객정보, 사내 데이터 등)의 유출 정황을 파악하는 것입니다.
1 본 분석은 프리렌서 개발팀 조민하 직원의 업무용 Windows PC에 대하여 정보를 수집하였으며, 행위에 따라 기록되는 주요 아티팩트 데이터를 확보하였습니다. 대상 기기 정보는 다음과 같습니다.

| MAC 주소 | PC 이름 | 운영체제 | IP 주소 |
|---|---|---|---|
| 00-0C-29-EB-B3-7B | DESKTOP-4L4O6MI | Windows 11 Version 24H2 | 192.168.74.135 |

수집된 아티팩트는 브라우저 사용 내역, USB 연결 기록, 프로그램 실행 기록(프리패치), 파일 실행 및 삭제 흔적, 메신저 이용 내역 등으로, 이를 종합적으로 분석하여 사용자의 활동을 확인할 수 있습니다.
2 분석은 자동화 시스템을 통해 이루어졌으며, 총 37초가 소요되었습니다.
분석 일시: 2025 년 9 월 23 일 화요일 오후 13:55 – 13:56
3 본 분석은 자동화된 시스템을 통해 아래 단계의 절차로 수행되었으며, 이를 통해 신속하고 일관된 결과를 도출하였습니다.
**1) 데이터 수집**
- 분석 시스템이 대상 PC에 접근하여 [1.2 데이터 수집]에 명시된 브라우저, 외부 저장 매체, 메신저 등 각 영역의 디지털 아티팩트 원본 데이터를 일괄 수집합니다.
**2) 데이터 전처리 및 필터링**
- 수집된 로그, 데이터베이스, 시스템 파일 등 비정형 데이터를 분석 가능한 정형 데이터로 변환합니다.
- 이후, 분석 목적과 관련 없는 시스템의 기본 동작 기록(쓰레기 데이터)을 제거하여 사용자의 유의미한 행위 데이터만을 추출합